In [2]:
from rdflib import Graph, Namespace, URIRef, RDF, RDFS, OWL, Literal, XSD


In [3]:
ontology_path = "Updated_Food_Health_sustainability_v6.ttl"
g = Graph()
g.parse(ontology_path, format="turtle")

<Graph identifier=N5005c04f3cc4441b8222afbe9e9c0aba (<class 'rdflib.graph.Graph'>)>

In [4]:
# Define the namespace
ex = Namespace("http://example.com#")

In [5]:
#Rule: No alcoholic beverages recommended for Pregnancy
# Get all instances of AlcoholicBeverages (i.e., individuals that belong to this class)
alcoholic_instances = set()
for s, p, o in g.triples((None, RDF.type, ex.AlcoholicBeverages)):
    alcoholic_instances.add(s)

# Check if we found any instances
if not alcoholic_instances:
    print("No instances of AlcoholicBeverages found. Check the class definition in the ontology.")
else:
    print(f"Found {len(alcoholic_instances)} instances of AlcoholicBeverages.")


# Define Pregnancy class URI
pregnancy_uri = ex.Pregnancy

# Apply the notRecommendedFor property to all instances
for beverage in alcoholic_instances:
    g.add((beverage, ex.notRecommendedFor, pregnancy_uri))

Found 41 instances of AlcoholicBeverages.


In [6]:
preg_string = "Preganncy"

# List of food groups not recommended for pregnancy
non_preg_food_groups = [
    ex.AlcoholicBeverages 
    
]

# Find all persons with hasHealthState containing "Pregnancy" as a literal value
for person, _, health_state in g.triples((None, ex.hasHealthState, None)):
    if isinstance(health_state, Literal) and health_state.lower() == preg_string.lower():  # Case-insensitive match
        # Assign notRecommendedFor Vegan to the food groups
        for food_group in non_preg_food_groups:
            g.add((food_group, ex.notRecommendedFor, person))
            print(f"✅ {food_group} is now notRecommendedFor {person} due to Pregnancy health state")


In [7]:
#Rule: No food containing lactose is recommended to Lactose intolerant people, and people following a lactose restricted diet.
# Find all instances of Dairy (i.e., individuals that belong to the Dairy class)
dairy_instances = set()

for s, p, o in g.triples((None, RDF.type, ex.Dairy)):
    dairy_instances.add(s)

# Check if instances were found
if not dairy_instances:
    print("No instances of Dairy found. Check the class definition in the ontology.")
else:
    print(f"Found {len(dairy_instances)} instances of Dairy.")

# Define LactoseIntolerant as an instance of Intolerance
lactose_intolerant = ex.LactoseIntolerant

# Apply the notRecommendedFor property to all instances of Dairy
for dairy in dairy_instances:
    g.add((dairy, ex.notRecommendedFor, lactose_intolerant))

Found 205 instances of Dairy.


In [8]:
#Rule: No food containing gluten is recommended to Gluten intolerant people, and people following a Gluten restricted diet.
# List of food instances that contain gluten
gluten_foods = [
    "Food_Bread", "Food_Pasta", "Food_beer_old_brown", "Food_WheatFlour", "Food_beer_white","Food_beer_w_fruit_flavor",
    "Food_Cookies", "Food_Cereal", "Food_Crackers", "Food_soy_sauce_based_on_ketjap","Food_soy_sauce_salt_ketjap", "Food_beer_w_fruitjuice_radler","Food_beer_bock",
    "Food_biscuit_salted"

]

# Define GlutenIntolerant as an instance of Intolerance
gluten_intolerant = ex.GlutenIntolerant

# Apply the notRecommendedFor property to all gluten-containing foods
for food in gluten_foods:
    food_uri = ex[food]
    g.add((food_uri, ex.notRecommendedFor, gluten_intolerant))

In [9]:
# Find all instances of Pastry_and_Biscuits (i.e., individuals of this class)
pastry_instances = set()

for s, p, o in g.triples((None, RDF.type, ex.Pastry_and_Biscuits)):
    pastry_instances.add(s)

# Check if instances were found
if not pastry_instances:
    print("No instances of Pastry_and_Biscuits found. Check the class definition in the ontology.")
else:
    print(f"Found {len(pastry_instances)} instances of Pastry_and_Biscuits.")

# Define GlutenIntolerant as an instance of Intolerance
gluten_intolerant = ex.GlutenIntolerant

# Apply the notRecommendedFor property to all instances of Pastry_and_Biscuits
for pastry in pastry_instances:
    g.add((pastry, ex.notRecommendedFor, gluten_intolerant))

Found 170 instances of Pastry_and_Biscuits.


In [10]:
#Rule:a person under 12 is a child, a person above 13 and under 17 is a teenager,
#a person above 18 and under 64 is an adult,a person above 64 is a senior adult,

# Define age categories
AGE_CATEGORIES = {
    "Child": (0, 12),
    "Teenager": (13, 17),
    "Adult": (18, 64),
    "SeniorAdult": (65, float("inf"))
}

# Find all persons who have an age and assign the correct category
for person, _, age in g.triples((None, ex.hasAge, None)):
    if isinstance(age, Literal):  
        try:
            age_value = int(age.value)  # Convert age to integer

            # Determine the age category
            for category, (min_age, max_age) in AGE_CATEGORIES.items():
                if min_age <= age_value <= max_age:
                    g.add((person, RDF.type, ex[category]))  # Assign type based on category
                    print(f"✅ Assigned {category} to {person} (Age: {age_value})")
                    break

        except ValueError:
            print(f"❌ Invalid age value for {person}: {age}")

✅ Assigned Adult to http://example.com#Person_1861 (Age: 49)
✅ Assigned Adult to http://example.com#Person_1873 (Age: 31)
✅ Assigned Adult to http://example.com#Person_5691 (Age: 31)
✅ Assigned Adult to http://example.com#Person_2161 (Age: 24)
✅ Assigned Adult to http://example.com#Person_4256 (Age: 24)
✅ Assigned Adult to http://example.com#Person_2534 (Age: 55)
✅ Assigned Adult to http://example.com#Person_5861 (Age: 55)
✅ Assigned Adult to http://example.com#Person_7473 (Age: 55)
✅ Assigned Adult to http://example.com#Person_2754 (Age: 42)
✅ Assigned Adult to http://example.com#Person_3064 (Age: 35)
✅ Assigned Adult to http://example.com#Person_3498 (Age: 19)
✅ Assigned Adult to http://example.com#Person_4004 (Age: 20)
✅ Assigned Adult to http://example.com#Person_4220 (Age: 23)
✅ Assigned Adult to http://example.com#Person_4304 (Age: 40)
✅ Assigned Adult to http://example.com#Person_4946 (Age: 40)
✅ Assigned Adult to http://example.com#Person_5720 (Age: 40)
✅ Assigned Adult to http

In [11]:
# Find all persons and classify them based on hasSex property
for person, _, sex in g.triples((None, ex.hasSex, None)):
    if isinstance(sex, Literal):
        if sex.lower() == "female":
            g.add((person, RDF.type, ex.Female))  # Assign Female subclass
        elif sex.lower() == "male":
            g.add((person, RDF.type, ex.Male))  # Assign Male subclass

In [12]:
# Find all persons with height and weight
for person, _, weight in g.triples((None, ex.hasWeight, None)):
    for _, _, height in g.triples((person, ex.hasHeight, None)):
        if isinstance(weight, Literal) and isinstance(height, Literal):
            try:
                # Convert values to float
                weight_kg = float(weight.value)
                height_value = float(height.value)

                # If height is in cm (e.g., > 10), convert to meters
                if height_value > 10:
                    height_m = height_value / 100  
                else:
                    height_m = height_value

                # Ensure height is valid before calculating BMI
                if height_m > 0:
                    bmi = round(weight_kg / (height_m ** 2), 2)  # BMI formula
                    g.add((person, ex.hasBMI, Literal(bmi, datatype=XSD.float)))
                    print(f"Calculated BMI for {person}: {bmi}")
                else:
                    print(f"Skipping {person}: Invalid height value ({height_value})")

            except ValueError:
                print(f"Invalid height/weight values for {person}")


Calculated BMI for http://example.com#Person_1861: 29.13
Calculated BMI for http://example.com#Person_1873: 39.54
Calculated BMI for http://example.com#Person_2161: 25.32
Calculated BMI for http://example.com#Person_2534: 39.24
Calculated BMI for http://example.com#Person_6729: 38.48
Calculated BMI for http://example.com#Person_2754: 23.95
Calculated BMI for http://example.com#Person_3064: 21.04
Calculated BMI for http://example.com#Person_3498: 33.98
Calculated BMI for http://example.com#Person_4004: 40.51
Calculated BMI for http://example.com#Person_4220: 33.89
Calculated BMI for http://example.com#Person_4256: 17.34
Calculated BMI for http://example.com#Person_5861: 19.06
Calculated BMI for http://example.com#Person_4304: 23.33
Calculated BMI for http://example.com#Person_4946: 25.42
Calculated BMI for http://example.com#Person_5449: 39.44
Calculated BMI for http://example.com#Person_5475: 25.9
Calculated BMI for http://example.com#Person_5669: 37.11
Calculated BMI for http://exampl

In [13]:
# Define BMI categories and their corresponding health states
bmi_categories = {
    "Underweight": (0, 18.5),
    "NormalWeight": (18.5, 25),
    "Overweight": (25, 30),
    "Obesity": (30, float("inf"))
}

# Find all persons with BMI and assign the appropriate health state
for person, _, bmi in g.triples((None, ex.hasBMI, None)):
    if isinstance(bmi, Literal):
        try:
            bmi_value = float(bmi)  # Convert RDF Literal to float

            # Determine the health state based on BMI
            for state, (min_bmi, max_bmi) in bmi_categories.items():
                if min_bmi <= bmi_value < max_bmi:
                    health_state_literal = Literal(state)  # Assign as string value

                    # Check if the person already has this health state assigned
                    if (person, ex.hasHealthState, health_state_literal) not in g:
                        g.add((person, ex.hasHealthState, health_state_literal))
                        print(f"✅ Assigned {state} to {person} (BMI: {bmi_value})")

                    break  # Exit loop after finding the correct category

        except ValueError:
            print(f"⚠️ Invalid BMI value for {person}: {bmi}")


✅ Assigned Overweight to http://example.com#Person_1861 (BMI: 29.13)
✅ Assigned Obesity to http://example.com#Person_1873 (BMI: 39.54)
✅ Assigned Overweight to http://example.com#Person_2161 (BMI: 25.32)
✅ Assigned Obesity to http://example.com#Person_2534 (BMI: 39.24)
✅ Assigned Obesity to http://example.com#Person_6729 (BMI: 38.48)
✅ Assigned NormalWeight to http://example.com#Person_2754 (BMI: 23.95)
✅ Assigned NormalWeight to http://example.com#Person_3064 (BMI: 21.04)
✅ Assigned Obesity to http://example.com#Person_3498 (BMI: 33.98)
✅ Assigned Obesity to http://example.com#Person_4004 (BMI: 40.51)
✅ Assigned Obesity to http://example.com#Person_4220 (BMI: 33.89)
✅ Assigned Underweight to http://example.com#Person_4256 (BMI: 17.34)
✅ Assigned NormalWeight to http://example.com#Person_5861 (BMI: 19.06)
✅ Assigned NormalWeight to http://example.com#Person_4304 (BMI: 23.33)
✅ Assigned Overweight to http://example.com#Person_4946 (BMI: 25.42)
✅ Assigned Obesity to http://example.com#Pe

In [14]:
glucose_restricted_diet = ex.GlucoseRestrictedDiet  # The diet individual

# Find all persons with hasHealthState containing "Diabetes" as a literal value
for person, _, health_state in g.triples((None, ex.hasHealthState, None)):
    if isinstance(health_state, Literal) and health_state.lower() == "diabetes":  # Case-insensitive match
        # Assign followsDiet = GlucoseRestrictedDiet (the individual)
        g.add((person, ex.followsDiet, glucose_restricted_diet))
        print(f"✅ {person} now follows {glucose_restricted_diet} due to Diabetes")

✅ http://example.com#Person_1861 now follows http://example.com#GlucoseRestrictedDiet due to Diabetes
✅ http://example.com#Person_4004 now follows http://example.com#GlucoseRestrictedDiet due to Diabetes
✅ http://example.com#Person_4304 now follows http://example.com#GlucoseRestrictedDiet due to Diabetes
✅ http://example.com#Person_5962 now follows http://example.com#GlucoseRestrictedDiet due to Diabetes


In [15]:
# Vegan as a string value in hasHealthState
vegan_string = "Vegan"

# Vegan as a subclass of Diet
vegan_diet_class = ex.Vegan  # Class that is a subclass of Diet

# List of food groups not recommended for vegans
non_vegan_food_groups = [
    ex.Dairy, 
    ex.Fish, 
    ex.Meat, 
    ex.MixedDishes, 
    ex.Pastry_and_Biscuits, 
    ex.SavouryDishes
]

# 1️⃣ Assign `notRecommendedFor` to persons who have `hasHealthState = "Vegan"`
for person, _, health_state in g.triples((None, ex.hasHealthState, None)):
    if isinstance(health_state, Literal) and health_state.lower() == vegan_string.lower():  # Case-insensitive match
        for food_group in non_vegan_food_groups:
            g.add((food_group, ex.notRecommendedFor, person))
            print(f"✅ {food_group} is now notRecommendedFor {person} due to Vegan health state")

# 2️⃣ Assign `notRecommendedFor` to `Vegan` as a Diet class
for food_group in non_vegan_food_groups:
    g.add((food_group, ex.notRecommendedFor, vegan_diet_class))
    print(f"✅ {food_group} is now notRecommendedFor {vegan_diet_class} as a Diet class")

✅ http://example.com#Dairy is now notRecommendedFor http://example.com#Person_9823 due to Vegan health state
✅ http://example.com#Fish is now notRecommendedFor http://example.com#Person_9823 due to Vegan health state
✅ http://example.com#Meat is now notRecommendedFor http://example.com#Person_9823 due to Vegan health state
✅ http://example.com#MixedDishes is now notRecommendedFor http://example.com#Person_9823 due to Vegan health state
✅ http://example.com#Pastry_and_Biscuits is now notRecommendedFor http://example.com#Person_9823 due to Vegan health state
✅ http://example.com#SavouryDishes is now notRecommendedFor http://example.com#Person_9823 due to Vegan health state
✅ http://example.com#Dairy is now notRecommendedFor http://example.com#Vegan as a Diet class
✅ http://example.com#Fish is now notRecommendedFor http://example.com#Vegan as a Diet class
✅ http://example.com#Meat is now notRecommendedFor http://example.com#Vegan as a Diet class
✅ http://example.com#MixedDishes is now not

In [16]:
# Vegetarian as a string value in hasHealthState
vegetarian_string = "Vegetarian"

# Vegetarian as a subclass of Diet
vegetarian_diet_class = ex.Vegetarian  # Class that is a subclass of Diet

# List of food groups not recommended for vegetarians
non_vegetarian_food_groups = [
    ex.Fish, 
    ex.Meat
]

# 1️⃣ Assign `notRecommendedFor` to persons who have `hasHealthState = "Vegetarian"`
for person, _, health_state in g.triples((None, ex.hasHealthState, None)):
    if isinstance(health_state, Literal) and health_state.lower() == vegetarian_string.lower():  # Case-insensitive match
        for food_group in non_vegetarian_food_groups:
            g.add((food_group, ex.notRecommendedFor, person))
            print(f"✅ {food_group} is now notRecommendedFor {person} due to Vegetarian health state")

# 2️⃣ Assign `notRecommendedFor` to `Vegetarian` as a Diet class
for food_group in non_vegetarian_food_groups:
    g.add((food_group, ex.notRecommendedFor, vegetarian_diet_class))
    print(f"✅ {food_group} is now notRecommendedFor {vegetarian_diet_class} as a Diet class")

✅ http://example.com#Fish is now notRecommendedFor http://example.com#Person_5475 due to Vegetarian health state
✅ http://example.com#Meat is now notRecommendedFor http://example.com#Person_5475 due to Vegetarian health state
✅ http://example.com#Fish is now notRecommendedFor http://example.com#Vegetarian as a Diet class
✅ http://example.com#Meat is now notRecommendedFor http://example.com#Vegetarian as a Diet class


In [17]:
# Pescatarian as a string value in hasHealthState
pescatarian_string = "Pescatarian"

# Pescatarian as a subclass of Diet
pescatarian_diet_class = ex.Pescatarian  # Class that is a subclass of Diet

# Food group not recommended for pescatarians (Meat only)
non_pescatarian_food_groups = [ex.Meat]

# 1️⃣ Assign `notRecommendedFor` to persons who have `hasHealthState = "Pescatarian"`
for person, _, health_state in g.triples((None, ex.hasHealthState, None)):
    if isinstance(health_state, Literal) and health_state.lower() == pescatarian_string.lower():  # Case-insensitive match
        for food_group in non_pescatarian_food_groups:
            g.add((food_group, ex.notRecommendedFor, person))
            print(f"✅ {food_group} is now notRecommendedFor {person} due to Pescatarian health state")

# 2️⃣ Assign `notRecommendedFor` to `Pescatarian` as a Diet class
for food_group in non_pescatarian_food_groups:
    g.add((food_group, ex.notRecommendedFor, pescatarian_diet_class))
    print(f"✅ {food_group} is now notRecommendedFor {pescatarian_diet_class} as a Diet class")


✅ http://example.com#Meat is now notRecommendedFor http://example.com#Pescatarian as a Diet class


In [18]:
#Meat (Such as Pork) Is notRecommendedFor Halal Diet
halal_diet_class = ex.HalalDiet  # Ensure this exists as a subclass

# List of pork-related keywords (adjust if necessary)
pork_keywords = ["pork", "bacon", "ham", "lard", "pig"]

# 1️⃣ Find all instances of the Meat class
for meat_instance, _, meat_class in g.triples((None, RDF.type, ex.Meat)):
    # Convert meat instance URI to string and check if it contains pork-related terms
    meat_name = str(meat_instance).lower()
    if any(keyword in meat_name for keyword in pork_keywords):
        # Assign notRecommendedFor HalalDiet (as a subclass of Diet)
        g.add((meat_instance, ex.notRecommendedFor, halal_diet_class))
        print(f"✅ {meat_instance} is now notRecommendedFor {halal_diet_class} due to being pork-based")

✅ http://example.com#Food_bacon is now notRecommendedFor http://example.com#HalalDiet due to being pork-based
✅ http://example.com#Food_bacon_fat_smoked_prepared is now notRecommendedFor http://example.com#HalalDiet due to being pork-based
✅ http://example.com#Food_bacon_fat_smoked_raw is now notRecommendedFor http://example.com#HalalDiet due to being pork-based
✅ http://example.com#Food_bacon_lean_prepared is now notRecommendedFor http://example.com#HalalDiet due to being pork-based
✅ http://example.com#Food_bacon_lean_smoked_prepared is now notRecommendedFor http://example.com#HalalDiet due to being pork-based
✅ http://example.com#Food_bacon_lean_smoked_raw is now notRecommendedFor http://example.com#HalalDiet due to being pork-based
✅ http://example.com#Food_bacon_rasher_prepared_in_own_fat is now notRecommendedFor http://example.com#HalalDiet due to being pork-based
✅ http://example.com#Food_bacon_rasher_raw is now notRecommendedFor http://example.com#HalalDiet due to being pork-ba

In [19]:
#Rule: High Fiber food not recommended for DietaryFiberRestrictedDiet
# Define DietaryFiberRestrictedDiet as a subclass of Diet
fiber_restricted_diet = ex.DietaryFiberRestrictedDiet

# Keywords related to high-fiber foods (adjust as needed)
high_fiber_keywords = [
    "wholegrain", "whole wheat", "bran", "oat", "quinoa", "barley", "bulgur",
    "lentil", "chickpea", "bean", "peas", "soy", "almond", "walnut", "cashew", 
    "chia", "flax", "sunflower", "coconut", "berries", "fig", "orange", 
    "carrot", "broccoli", "corn", "celery", "cucumber", "raisins", "dates", "prune"
]

# Find all food instances in the ontology
for food_instance, _, food_class in g.triples((None, RDF.type, ex.Food)):  # Assuming all foods are under `ex:Food`
    food_name = str(food_instance).lower()  # Convert URI to lowercase string
    if any(keyword in food_name for keyword in high_fiber_keywords):
        # Assign `notRecommendedFor` to DietaryFiberRestrictedDiet
        g.add((food_instance, ex.notRecommendedFor, fiber_restricted_diet))
        print(f"✅ {food_instance} is now notRecommendedFor {fiber_restricted_diet} due to high fiber content")

✅ http://example.com#Food_almond_imitation_paste_av is now notRecommendedFor http://example.com#DietaryFiberRestrictedDiet due to high fiber content
✅ http://example.com#Food_almond_paste_filled_tarts_av is now notRecommendedFor http://example.com#DietaryFiberRestrictedDiet due to high fiber content
✅ http://example.com#Food_almond_paste_filled_tarts_w_butter is now notRecommendedFor http://example.com#DietaryFiberRestrictedDiet due to high fiber content
✅ http://example.com#Food_almond_paste_filled_tarts_wo_butter is now notRecommendedFor http://example.com#DietaryFiberRestrictedDiet due to high fiber content
✅ http://example.com#Food_almond_paste_w_egg is now notRecommendedFor http://example.com#DietaryFiberRestrictedDiet due to high fiber content
✅ http://example.com#Food_almonds_blanched_salted is now notRecommendedFor http://example.com#DietaryFiberRestrictedDiet due to high fiber content
✅ http://example.com#Food_almonds_blanched_unsalted is now notRecommendedFor http://example.c

In [20]:
#Rule: High Fat food not recommended for FatRestrictedDiet
# Define FatRestrictedDiet as a subclass of Diet
fat_restricted_diet = ex.FatRestrictedDiet

# Keywords related to high-fat foods (adjust as needed)
high_fat_keywords = [
    "fried", "butter", "cheese", "cream", "whole milk", "pork", "bacon", "sausage",
    "salami", "pepperoni", "lamb", "duck", "hot dog", "mayonnaise", "margarine",
    "chocolate", "pastry", "cookies", "ice cream", "whipped", "fudge", "peanut butter",
    "almonds", "cashews", "macadamia", "coconut", "shortening", "lard"
]

# Find all food instances in the ontology
for food_instance, _, food_class in g.triples((None, RDF.type, ex.Food)):  # Assuming all foods are under `ex:Food`
    food_name = str(food_instance).lower()  # Convert URI to lowercase string
    if any(keyword in food_name for keyword in high_fat_keywords):
        # Assign `notRecommendedFor` to FatRestrictedDiet
        g.add((food_instance, ex.notRecommendedFor, fat_restricted_diet))
        print(f"✅ {food_instance} is now notRecommendedFor {fat_restricted_diet} due to high fat content")

✅ http://example.com#Food_after_eight_chocolate_mints is now notRecommendedFor http://example.com#FatRestrictedDiet due to high fat content
✅ http://example.com#Food_almond_paste_filled_tarts_w_butter is now notRecommendedFor http://example.com#FatRestrictedDiet due to high fat content
✅ http://example.com#Food_almond_paste_filled_tarts_wo_butter is now notRecommendedFor http://example.com#FatRestrictedDiet due to high fat content
✅ http://example.com#Food_almonds_blanched_salted is now notRecommendedFor http://example.com#FatRestrictedDiet due to high fat content
✅ http://example.com#Food_almonds_blanched_unsalted is now notRecommendedFor http://example.com#FatRestrictedDiet due to high fat content
✅ http://example.com#Food_almonds_w_skin_salted is now notRecommendedFor http://example.com#FatRestrictedDiet due to high fat content
✅ http://example.com#Food_almonds_w_skin_unsalted is now notRecommendedFor http://example.com#FatRestrictedDiet due to high fat content
✅ http://example.com#

In [21]:
#Rule: Restricted Food for a LactoseRestrictedDiet
# Define LactoseRestrictedDiet as a subclass of Diet
lactose_restricted_diet = ex.LactoseRestrictedDiet

# Keywords related to high-lactose foods (adjust as needed)
high_lactose_keywords = [
    "milk", "cream", "cheese", "butter", "yogurt", "ice cream", "custard", 
    "whey", "casein", "chocolate", "pudding", "sherbet", "cottage", "ricotta", 
    "cheesy", "alfredo", "croissant", "biscuit", "latte", "mocha"
]

# Find all food instances in the ontology
for food_instance, _, food_class in g.triples((None, RDF.type, ex.Food)):  # Assuming all foods are under `ex:Food`
    food_name = str(food_instance).lower()  # Convert URI to lowercase string
    if any(keyword in food_name for keyword in high_lactose_keywords):
        # Assign `notRecommendedFor` to LactoseRestrictedDiet
        g.add((food_instance, ex.notRecommendedFor, lactose_restricted_diet))
        print(f"✅ {food_instance} is now notRecommendedFor {lactose_restricted_diet} due to lactose content")

✅ http://example.com#Food_after_eight_chocolate_mints is now notRecommendedFor http://example.com#LactoseRestrictedDiet due to lactose content
✅ http://example.com#Food_almond_paste_filled_tarts_w_butter is now notRecommendedFor http://example.com#LactoseRestrictedDiet due to lactose content
✅ http://example.com#Food_almond_paste_filled_tarts_wo_butter is now notRecommendedFor http://example.com#LactoseRestrictedDiet due to lactose content
✅ http://example.com#Food_appel_pie_dutch_w_shortbread_w_butter is now notRecommendedFor http://example.com#LactoseRestrictedDiet due to lactose content
✅ http://example.com#Food_apple_pie_dutch_w_shortbread_wo_butter is now notRecommendedFor http://example.com#LactoseRestrictedDiet due to lactose content
✅ http://example.com#Food_apple_turnover_w_puff_pastry_w_butter is now notRecommendedFor http://example.com#LactoseRestrictedDiet due to lactose content
✅ http://example.com#Food_apple_turnover_w_puff_pastry_wo_butter is now notRecommendedFor http:/

In [22]:
#Rule: food notRecommendedFor a lightDigestibleDiet
# Define LightDigestibleDiet as a subclass of Diet
light_digestible_diet = ex.LightDigestibleDiet

# Keywords related to hard-to-digest foods (adjust as needed)
hard_to_digest_keywords = [
    "fried", "fatty", "spicy", "hot", "chili", "salsa", "tomato", "vinegar", "citrus", "orange",
    "cabbage", "broccoli", "cauliflower", "brussels", "lentil", "beans", "peas", "corn", "garlic",
    "whole wheat", "quinoa", "bran", "oatmeal", "granola", "cheese", "cream", "ice cream", "yogurt",
    "soda", "coffee", "black tea", "alcohol", "chocolate", "candy", "sorbitol", "xylitol"
]

# Find all food instances in the ontology
for food_instance, _, food_class in g.triples((None, RDF.type, ex.Food)):  # Assuming all foods are under `ex:Food`
    food_name = str(food_instance).lower()  # Convert URI to lowercase string
    if any(keyword in food_name for keyword in hard_to_digest_keywords):
        # Assign `notRecommendedFor` to LightDigestibleDiet
        g.add((food_instance, ex.notRecommendedFor, light_digestible_diet))
        print(f"✅ {food_instance} is now notRecommendedFor {light_digestible_diet} due to digestion difficulty")

✅ http://example.com#Food_after_eight_chocolate_mints is now notRecommendedFor http://example.com#LightDigestibleDiet due to digestion difficulty
✅ http://example.com#Food_baked_chicory_w_cheese_and_ham is now notRecommendedFor http://example.com#LightDigestibleDiet due to digestion difficulty
✅ http://example.com#Food_bakery_mix_for_chocolate_cake is now notRecommendedFor http://example.com#LightDigestibleDiet due to digestion difficulty
✅ http://example.com#Food_beans_black_canned is now notRecommendedFor http://example.com#LightDigestibleDiet due to digestion difficulty
✅ http://example.com#Food_beans_black_eyed_canned is now notRecommendedFor http://example.com#LightDigestibleDiet due to digestion difficulty
✅ http://example.com#Food_beans_black_eyed_dried is now notRecommendedFor http://example.com#LightDigestibleDiet due to digestion difficulty
✅ http://example.com#Food_beans_broad_boiled is now notRecommendedFor http://example.com#LightDigestibleDiet due to digestion difficulty


In [23]:
#Rule: Food notRecommendedFor PregnancyRestrictedDiet Using Keywords
# Define PregnancyRestrictedDiet as a subclass of Diet
pregnancy_restricted_diet = ex.PregnancyRestrictedDiet

# Define Pregnant as a subclass of HealthState
pregnant_healthstate_class = ex.Pregnant

# Keywords related to foods unsafe for pregnancy (adjust as needed)
pregnancy_risky_keywords = [
    "raw", "undercooked", "unpasteurized", "caffeine", "alcohol", "soft cheese", "shark", "swordfish", 
    "mackerel", "bigeye tuna", "marlin", "sprouts", "energy drink", "beer", "wine", "liquor", "soda",
    "sashimi", "oysters", "clams", "pâté", "cold cuts", "salami", "pepperoni", "hot dog", "processed meat",
    "brie", "camembert", "roquefort", "feta", "queso fresco", "cookie dough", "high sugar", "diet soda"
]

# 1️⃣ Find all persons with hasHealthState = "Pregnant"
pregnant_string = "Pregnant"
pregnant_persons = set()

for person, _, health_state in g.triples((None, ex.hasHealthState, None)):
    if isinstance(health_state, Literal) and health_state.lower() == pregnant_string.lower():  # Case-insensitive match
        pregnant_persons.add(person)
        print(f"👶 {person} identified as Pregnant")

# 2️⃣ Find all food instances in the ontology and apply `notRecommendedFor`
for food_instance, _, food_class in g.triples((None, RDF.type, ex.Food)):  # Assuming all foods are under `ex:Food`
    food_name = str(food_instance).lower()  # Convert URI to lowercase string
    if any(keyword in food_name for keyword in pregnancy_risky_keywords):
        # Assign `notRecommendedFor` to PregnancyRestrictedDiet
        g.add((food_instance, ex.notRecommendedFor, pregnancy_restricted_diet))
        print(f"✅ {food_instance} is now notRecommendedFor {pregnancy_restricted_diet} due to pregnancy risk")

        # Also assign `notRecommendedFor` for persons who are Pregnant
        for pregnant_person in pregnant_persons:
            g.add((food_instance, ex.notRecommendedFor, pregnant_person))
            print(f"⚠️ {food_instance} is now notRecommendedFor {pregnant_person}")

        # 3️⃣ Assign `notRecommendedFor` to Pregnant as a subclass of HealthState
        g.add((food_instance, ex.notRecommendedFor, pregnant_healthstate_class))
        print(f"🚫 {food_instance} is now notRecommendedFor {pregnant_healthstate_class} as a HealthState subclass")

✅ http://example.com#Food_amaranth_leaves_raw is now notRecommendedFor http://example.com#PregnancyRestrictedDiet due to pregnancy risk
🚫 http://example.com#Food_amaranth_leaves_raw is now notRecommendedFor http://example.com#Pregnant as a HealthState subclass
✅ http://example.com#Food_anchovy_raw is now notRecommendedFor http://example.com#PregnancyRestrictedDiet due to pregnancy risk
🚫 http://example.com#Food_anchovy_raw is now notRecommendedFor http://example.com#Pregnant as a HealthState subclass
✅ http://example.com#Food_antroewa_raw is now notRecommendedFor http://example.com#PregnancyRestrictedDiet due to pregnancy risk
🚫 http://example.com#Food_antroewa_raw is now notRecommendedFor http://example.com#Pregnant as a HealthState subclass
✅ http://example.com#Food_artichoke_raw is now notRecommendedFor http://example.com#PregnancyRestrictedDiet due to pregnancy risk
🚫 http://example.com#Food_artichoke_raw is now notRecommendedFor http://example.com#Pregnant as a HealthState subclas

In [24]:
#Rule: Foods notRecommendedFor a SaltRestrictedDiet
# Define SaltRestrictedDiet as a subclass of Diet
salt_restricted_diet = ex.SaltRestrictedDiet

# Keywords related to high-sodium foods (adjust as needed)
high_sodium_keywords = [
    "salted", "cured", "pickled", "processed", "soy sauce", "teriyaki", "barbecue",
    "bacon", "ham", "salami", "pepperoni", "deli", "sausages", "hot dog", "smoked", 
    "canned", "instant", "ramen", "bouillon", "cheese", "snack", "crackers", "fries", 
    "breadsticks", "stuffing", "club soda", "tonic", "gravy", "seasoned", "pretzel"
]

# Find all food instances in the ontology
for food_instance, _, food_class in g.triples((None, RDF.type, ex.Food)):  # Assuming all foods are under `ex:Food`
    food_name = str(food_instance).lower()  # Convert URI to lowercase string
    if any(keyword in food_name for keyword in high_sodium_keywords):
        # Assign `notRecommendedFor` to SaltRestrictedDiet
        g.add((food_instance, ex.notRecommendedFor, salt_restricted_diet))
        print(f"✅ {food_instance} is now notRecommendedFor {salt_restricted_diet} due to high sodium content")


✅ http://example.com#Food_almonds_blanched_salted is now notRecommendedFor http://example.com#SaltRestrictedDiet due to high sodium content
✅ http://example.com#Food_almonds_blanched_unsalted is now notRecommendedFor http://example.com#SaltRestrictedDiet due to high sodium content
✅ http://example.com#Food_almonds_w_skin_salted is now notRecommendedFor http://example.com#SaltRestrictedDiet due to high sodium content
✅ http://example.com#Food_almonds_w_skin_unsalted is now notRecommendedFor http://example.com#SaltRestrictedDiet due to high sodium content
✅ http://example.com#Food_anchovy_in_oil_canned is now notRecommendedFor http://example.com#SaltRestrictedDiet due to high sodium content
✅ http://example.com#Food_artichoke_hearts_canned is now notRecommendedFor http://example.com#SaltRestrictedDiet due to high sodium content
✅ http://example.com#Food_bacon is now notRecommendedFor http://example.com#SaltRestrictedDiet due to high sodium content
✅ http://example.com#Food_bacon_fat_smok

In [25]:
#Rule: Food RecommendedFor to Anemia
# Define Anemia as a subclass of HealthState
anemia_health_state = ex.Anemia

# Keywords related to foods recommended for anemia (adjust as needed)
anemia_friendly_keywords = [
    "beef", "liver", "turkey", "chicken", "pork", "seafood", "oysters", "clams", "mussels", "sardines",
    "lentils", "chickpeas", "kidney beans", "black beans", "tofu", "tempeh", "pumpkin seeds",
    "sesame seeds", "tahini", "cashews", "walnuts", "quinoa", "fortified", "citrus", "orange", "bell pepper",
    "tomato", "strawberry", "kiwi", "guava", "kale", "spinach", "avocado", "b12", "folic acid", "egg", "red meat"
]

# Find all food instances in the ontology
for food_instance, _, food_class in g.triples((None, RDF.type, ex.Food)):  # Assuming all foods are under `ex:Food`
    food_name = str(food_instance).lower()  # Convert URI to lowercase string
    if any(keyword in food_name for keyword in anemia_friendly_keywords):
        # Assign `RecommendedFor` to Anemia
        g.add((food_instance, ex.RecommendedFor, anemia_health_state))
        print(f"✅ {food_instance} is now RecommendedFor {anemia_health_state} due to high iron or vitamin content")

✅ http://example.com#Food_almond_paste_w_egg is now RecommendedFor http://example.com#Anemia due to high iron or vitamin content
✅ http://example.com#Food_aubergine_eggplant_boiled is now RecommendedFor http://example.com#Anemia due to high iron or vitamin content
✅ http://example.com#Food_aubergine_eggplant_raw is now RecommendedFor http://example.com#Anemia due to high iron or vitamin content
✅ http://example.com#Food_avocado is now RecommendedFor http://example.com#Anemia due to high iron or vitamin content
✅ http://example.com#Food_beans_white_baked_in_tomato_sauce_canned is now RecommendedFor http://example.com#Anemia due to high iron or vitamin content
✅ http://example.com#Food_beef__10__fat_prepared_av is now RecommendedFor http://example.com#Anemia due to high iron or vitamin content
✅ http://example.com#Food_beef__5__fat_raw_av is now RecommendedFor http://example.com#Anemia due to high iron or vitamin content
✅ http://example.com#Food_beef_av_raw is now RecommendedFor http://

In [26]:
#Rule: Food notRecommendedFor to Anemia
# Define Anemia as a subclass of HealthState
anemia_health_state = ex.Anemia

# Keywords related to foods that block iron absorption (adjust as needed)
anemia_blocking_keywords = [
    "milk", "cheese", "yogurt", "cream", "butter", "calcium", "sardines", "coffee", "tea", "caffeine",
    "chocolate", "whole wheat", "brown rice", "lentils", "chickpeas", "black beans", "soy", "spinach",
    "rhubarb", "swiss chard", "peanut", "walnut", "almond", "candy", "soda", "artificial sweetener",
    "processed meat", "pickles", "fast food", "beer", "wine", "liquor", "salt", "canned soup"
]

# Find all food instances in the ontology
for food_instance, _, food_class in g.triples((None, RDF.type, ex.Food)):  # Assuming all foods are under `ex:Food`
    food_name = str(food_instance).lower()  # Convert URI to lowercase string
    if any(keyword in food_name for keyword in anemia_blocking_keywords):
        # Assign `notRecommendedFor` to Anemia
        g.add((food_instance, ex.notRecommendedFor, anemia_health_state))
        print(f"🚫 {food_instance} is now notRecommendedFor {anemia_health_state} due to iron-blocking properties")


🚫 http://example.com#Food_after_eight_chocolate_mints is now notRecommendedFor http://example.com#Anemia due to iron-blocking properties
🚫 http://example.com#Food_almond_imitation_paste_av is now notRecommendedFor http://example.com#Anemia due to iron-blocking properties
🚫 http://example.com#Food_almond_paste_filled_tarts_av is now notRecommendedFor http://example.com#Anemia due to iron-blocking properties
🚫 http://example.com#Food_almond_paste_filled_tarts_w_butter is now notRecommendedFor http://example.com#Anemia due to iron-blocking properties
🚫 http://example.com#Food_almond_paste_filled_tarts_wo_butter is now notRecommendedFor http://example.com#Anemia due to iron-blocking properties
🚫 http://example.com#Food_almond_paste_w_egg is now notRecommendedFor http://example.com#Anemia due to iron-blocking properties
🚫 http://example.com#Food_almonds_blanched_salted is now notRecommendedFor http://example.com#Anemia due to iron-blocking properties
🚫 http://example.com#Food_almonds_blanch

In [27]:
#Rules:food RecommendedFor Diabetes, and notRecommendedFor Diabetes
# Define Diabetes as a subclass of HealthState
diabetes_health_state = ex.Diabetes

# Keywords related to foods recommended for diabetes
diabetes_friendly_keywords = [
    "quinoa", "brown rice", "whole grain", "oatmeal", "barley", "buckwheat",
    "broccoli", "spinach", "kale", "swiss chard", "brussels sprouts", "cauliflower",
    "asparagus", "bell pepper", "cucumber", "eggplant", "salmon", "tuna", "mackerel",
    "tofu", "tempeh", "edamame", "lentils", "walnuts", "almonds", "flaxseeds",
    "chia", "avocado", "greek yogurt", "blueberries", "strawberries", "pears",
    "cherries", "cinnamon", "turmeric", "ginger", "green tea"
]

# Find all food instances in the ontology
for food_instance, _, food_class in g.triples((None, RDF.type, ex.Food)):
    food_name = str(food_instance).lower()
    if any(keyword in food_name for keyword in diabetes_friendly_keywords):
        # Assign `RecommendedFor` to Diabetes
        g.add((food_instance, ex.RecommendedFor, diabetes_health_state))
        print(f"✅ {food_instance} is now RecommendedFor {diabetes_health_state}")

✅ http://example.com#Food_almonds_blanched_salted is now RecommendedFor http://example.com#Diabetes
✅ http://example.com#Food_almonds_blanched_unsalted is now RecommendedFor http://example.com#Diabetes
✅ http://example.com#Food_almonds_w_skin_salted is now RecommendedFor http://example.com#Diabetes
✅ http://example.com#Food_almonds_w_skin_unsalted is now RecommendedFor http://example.com#Diabetes
✅ http://example.com#Food_asparagus_green_boiled is now RecommendedFor http://example.com#Diabetes
✅ http://example.com#Food_asparagus_green_raw is now RecommendedFor http://example.com#Diabetes
✅ http://example.com#Food_asparagus_white_boiled is now RecommendedFor http://example.com#Diabetes
✅ http://example.com#Food_asparagus_white_raw is now RecommendedFor http://example.com#Diabetes
✅ http://example.com#Food_asparagus_white_tinned is now RecommendedFor http://example.com#Diabetes
✅ http://example.com#Food_aubergine_eggplant_boiled is now RecommendedFor http://example.com#Diabetes
✅ http://

In [28]:
#Rule: Assigning Diets Based on HealthState 
# Dictionary mapping HealthState to Recommended Diet
health_diet_mapping = {
    "Hypertension": ex.SaltRestrictedDiet,
    "Diabetes": ex.LowCarbDiet,  # Or DiabetesDiet if defined
    "Anemia": ex.IronRichDiet,
    "Obesity": ex.FatRestrictedDiet,
    "Overweight": ex.LightDigestibleDiet,
    "Pregnancy": ex.PregnancyRestrictedDiet,
    "Intolerance": None,  # Depends on specific intolerance
    "FoodAllergy": None  # Depends on specific allergy
}

# Find all persons and check their HealthState
for person, _, health_state in g.triples((None, ex.hasHealthState, None)):
    health_state_str = str(health_state).split("#")[-1]  # Extract class name

    if health_state_str in health_diet_mapping and health_diet_mapping[health_state_str] is not None:
        # Assign `followsDiet` based on HealthState
        diet_to_follow = health_diet_mapping[health_state_str]
        g.add((person, ex.followsDiet, diet_to_follow))
        print(f"✅ {person} now follows {diet_to_follow} due to {health_state_str}")

✅ http://example.com#Person_1861 now follows http://example.com#LowCarbDiet due to Diabetes
✅ http://example.com#Person_4004 now follows http://example.com#LowCarbDiet due to Diabetes
✅ http://example.com#Person_4304 now follows http://example.com#LowCarbDiet due to Diabetes
✅ http://example.com#Person_5962 now follows http://example.com#LowCarbDiet due to Diabetes
✅ http://example.com#Person_2534 now follows http://example.com#IronRichDiet due to Anemia
✅ http://example.com#Person_5720 now follows http://example.com#IronRichDiet due to Anemia
✅ http://example.com#Person_5988 now follows http://example.com#IronRichDiet due to Anemia
✅ http://example.com#Person_8131 now follows http://example.com#IronRichDiet due to Anemia
✅ http://example.com#Person_3498 now follows http://example.com#LightDigestibleDiet due to Overweight
✅ http://example.com#Person_4220 now follows http://example.com#LightDigestibleDiet due to Overweight
✅ http://example.com#Person_5669 now follows http://example.com#

In [29]:
#recommended daily intake levels for these data properties based on  person's age, bmi, sex, and PerfomedLevelOfActivity
# Physical Activity Level (PAL) multipliers
PAL_VALUES = {
    "Sedentary": 1.3,
    "Lightly Active": 1.5,
    "Moderately Active": 1.7,
    "Very Active": 2.0
}

# WHO Micronutrient Recommendations (including Fiber)
MICRONUTRIENTS = {
    "VitaminA": {"male": 900, "female": 700},  # mcg
    "VitaminB6": {"male": 1.7, "female": 1.3},  # mg
    "VitaminD": {"male": 15, "female": 15},  # mcg
    "VitaminE": {"male": 15, "female": 15},  # mg
    "Iron": {"male": 8, "female": 18},  # mg
    "Calcium": {"male": 1000, "female": 1000},  # mg
    "Magnesium": {"male": 400, "female": 310},  # mg
    "Zinc": {"male": 11, "female": 8},  # mg
    "Sodium": {"male": 2000, "female": 2000},  # mg
    "Mercury": {"male": "Minimal", "female": "Minimal"}  # mg (Avoid high-mercury fish)
}

# WHO Fiber Recommendations
FIBER_RECOMMENDATIONS = {
    "male": {"Sedentary": 30, "Moderate": 35, "Active": 40},
    "female": {"Sedentary": 25, "Moderate": 28, "Active": 35},
}

# Macronutrient percentage ranges (WHO recommendations)
MACRO_RANGES = {
    "Protein": (10, 35),  # 10-35% of total calories
    "Fat": (20, 35),  # 20-35% of total calories
    "Carbohydrates": (45, 65)  # 45-65% of total calories
}

# Function to calculate BMR (Basal Metabolic Rate) using Schofield Equation
def calculate_bmr(sex, weight, height, age):
    if sex.lower() == "male":
        if age < 30:
            return 15.3 * weight + 679
        elif age < 60:
            return 11.6 * weight + 879
        else:
            return 13.5 * weight + 487
    else:
        if age < 30:
            return 14.7 * weight + 496
        elif age < 60:
            return 8.7 * weight + 829
        else:
            return 10.5 * weight + 596

# Function to calculate recommended daily intake
def calculate_daily_intake(person):
    # Retrieve person's attributes from ontology
    age = float(g.value(person, ex.hasAge))
    weight = float(g.value(person, ex.hasWeight))
    height = float(g.value(person, ex.hasHeight))
    sex = str(g.value(person, ex.hasSex)).lower()
    activity = str(g.value(person, ex.hasPerformedLevelOfActivity))
    weight_goal = str(g.value(person, ex.hasWeightGoal))  # Lose, Maintain, Gain Muscle

    # Calculate BMR
    bmr = calculate_bmr(sex, weight, height, age)

    # Adjust for activity level
    activity_multiplier = PAL_VALUES.get(activity, 1.3)  # Default to Sedentary
    daily_calories = bmr * activity_multiplier

    # Adjust for weight goal
    if weight_goal.lower() == "lose weight":
        daily_calories -= 500  # Deficit of 500 kcal/day
    elif weight_goal.lower() == "gain muscle":
        daily_calories += 500  # Surplus of 500 kcal/day

    # Calculate macronutrient distribution
    protein_range = (daily_calories * MACRO_RANGES["Protein"][0] / 400, daily_calories * MACRO_RANGES["Protein"][1] / 400)
    fat_range = (daily_calories * MACRO_RANGES["Fat"][0] / 900, daily_calories * MACRO_RANGES["Fat"][1] / 900)
    carb_range = (daily_calories * MACRO_RANGES["Carbohydrates"][0] / 400, daily_calories * MACRO_RANGES["Carbohydrates"][1] / 400)

    # Assign calculated values to ontology
    g.add((person, ex.AmountOfCaloriesNeededPerDay, Literal(daily_calories)))
    g.add((person, ex.AmountOfProteinNeededPerDay, Literal(f"{protein_range[0]:.1f}-{protein_range[1]:.1f} g")))
    g.add((person, ex.AmountOfFatNeededPerDay, Literal(f"{fat_range[0]:.1f}-{fat_range[1]:.1f} g")))
    g.add((person, ex.AmountOfCarbsNeededPerDay, Literal(f"{carb_range[0]:.1f}-{carb_range[1]:.1f} g")))

    # Add Fiber Needs
    fiber_needed = FIBER_RECOMMENDATIONS.get(sex, {}).get(activity, 25)  # Default 25g
    g.add((person, ex.AmountOfFiberNeededPerDay, Literal(f"{fiber_needed} g")))

    # Add Micronutrient Needs
    for nutrient, values in MICRONUTRIENTS.items():
        needed_amount = values.get(sex, "Unknown")
        g.add((person, getattr(ex, f"AmountOf{nutrient}NeededPerDay"), Literal(needed_amount)))

    print(f"✅ Updated daily intake for {person}")

# Apply function to all persons in ontology
for person, _, _ in g.triples((None, RDF.type, ex.Person)):
    calculate_daily_intake(person)


✅ Updated daily intake for http://example.com#Person_1861
✅ Updated daily intake for http://example.com#Person_1873
✅ Updated daily intake for http://example.com#Person_2161
✅ Updated daily intake for http://example.com#Person_2534
✅ Updated daily intake for http://example.com#Person_2754
✅ Updated daily intake for http://example.com#Person_3064
✅ Updated daily intake for http://example.com#Person_3498
✅ Updated daily intake for http://example.com#Person_4004
✅ Updated daily intake for http://example.com#Person_4220
✅ Updated daily intake for http://example.com#Person_4256
✅ Updated daily intake for http://example.com#Person_4304
✅ Updated daily intake for http://example.com#Person_4946
✅ Updated daily intake for http://example.com#Person_5449
✅ Updated daily intake for http://example.com#Person_5475
✅ Updated daily intake for http://example.com#Person_5669
✅ Updated daily intake for http://example.com#Person_5691
✅ Updated daily intake for http://example.com#Person_5720
✅ Updated dail

In [30]:
#defining hasNutrientScore of the foods
# Define Nutrients to Encourage (NR9)
NR9_NUTRIENTS = [
    "AmountOfProtein", "AmountOfFiber", "AmountOfVitaminA", "AmountOfVitaminC",
    "AmountOfVitaminD", "AmountOfCalcium", "AmountOfIron", "AmountOfMagnesium", "AmountOfPotassium"
]

# Define Nutrients to Limit (LIM3)
LIM3_NUTRIENTS = [
    "AmountOfSodium", "AmountOfSugar", "AmountOfSaturatedFat"
]

# Corrected: Use RDFS subClassOf instead of OWL.subClassOf
RDFS_SUBCLASS = URIRef("http://www.w3.org/2000/01/rdf-schema#subClassOf")

# Retrieve all subclasses of FoodGroup (Fixed)
food_subclasses = set()
for subclass, _, _ in g.triples((None, RDFS_SUBCLASS, ex.FoodGroup)):
    food_subclasses.add(subclass)

# Retrieve all instances of these subclasses
food_instances = set()
for subclass in food_subclasses:
    for instance, _, _ in g.triples((None, RDF.type, subclass)):
        food_instances.add(instance)

# Function to compute NRF score
def compute_nrf_score(food):
    nr9_sum = 0  # Total of beneficial nutrients
    lim3_sum = 0  # Total of limiting nutrients
    calories = g.value(food, ex.hasAmountOfCalories, None)

    if calories is None or float(calories) == 0:
        return None  # Avoid division by zero

    # Sum all beneficial nutrients
    for nutrient in NR9_NUTRIENTS:
        nutrient_value = g.value(food, getattr(ex, nutrient), None)
        if nutrient_value is not None:
            nr9_sum += float(nutrient_value)

    # Sum all limiting nutrients
    for nutrient in LIM3_NUTRIENTS:
        nutrient_value = g.value(food, getattr(ex, nutrient), None)
        if nutrient_value is not None:
            lim3_sum += float(nutrient_value)

    # Compute NRF score
    nrf_score = (nr9_sum - lim3_sum) / float(calories)
    return round(nrf_score, 2)

# Compute NRF Score for Each Food Instance
for food in food_instances:
    score = compute_nrf_score(food)
    if score is not None:
        g.add((food, ex.hasNutrientScore, Literal(score)))
        print(f"✅ {food} assigned a Nutrient Score of {score}")


✅ http://example.com#Food_cod_fillet_fried_simmered assigned a Nutrient Score of -0.57
✅ http://example.com#Food_sausage_dry_sucuk_turkish assigned a Nutrient Score of -3.34
✅ http://example.com#Food_lamb_chop_prepared assigned a Nutrient Score of -0.02
✅ http://example.com#Food_lemon_sole_raw assigned a Nutrient Score of -0.98
✅ http://example.com#Food_veal_frying_steak_prepared assigned a Nutrient Score of -0.24
✅ http://example.com#Food_cucumber_wo_skin_raw assigned a Nutrient Score of 2.11
✅ http://example.com#Food_herring_raw assigned a Nutrient Score of 0.35
✅ http://example.com#Food_minced_beef_kofte_turkish_prepared assigned a Nutrient Score of -2.84
✅ http://example.com#Food_wheat_rye_bread_wholemeal assigned a Nutrient Score of -1.14
✅ http://example.com#Food_chicken_wo_skin_grilled assigned a Nutrient Score of -0.28
✅ http://example.com#Food_lamb_leg_prepared assigned a Nutrient Score of -0.09
✅ http://example.com#Food_lettuce_romaine_raw assigned a Nutrient Score of 2.71
✅ 

In [31]:
#FoodInLimitedAmount & FoodToBeConsumedInHighAmount for Diabetes
 # Define keyword-based food categories for Diabetes (Matching Food Groups)
LIMITED_KEYWORDS_DIABETES = [
    "white bread", "white rice", "pastries", "sugary cereal", "sugar", "sweet", "candy",
    "soda", "juice", "flavored yogurt", "butter", "fried", "fast food", "chips", "cookies", "cake"
]

HIGH_AMOUNT_KEYWORDS_DIABETES = [
    "whole grains", "brown rice", "quinoa", "oats", "berries", "apple", "pear", "citrus",
    "greek yogurt", "low-fat dairy", "olive oil", "avocado", "nuts", "hummus", "seeds", "dark chocolate"
]

# Ensure Diabetes is defined as a subclass of HealthState
RDFS_SUBCLASS = URIRef("http://www.w3.org/2000/01/rdf-schema#subClassOf")
diabetes_class = None

for subclass, _, parent in g.triples((None, RDFS_SUBCLASS, ex.HealthState)):
    if str(subclass).endswith("Diabetes"):
        diabetes_class = subclass
        break

# Retrieve all FoodGroup subclasses
food_groups = set()
for subclass, _, parent in g.triples((None, RDFS_SUBCLASS, ex.FoodGroup)):
    food_groups.add(subclass)

# Find food instances and their FoodGroup assignments
food_instances = set()
for food, _, _ in g.triples((None, RDF.type, None)):
    # Check if this type is a FoodGroup subclass
    for food_group in food_groups:
        if (food, RDF.type, food_group) in g:
            food_instances.add((food, food_group))

# Assign food items based on FoodGroup matching
if diabetes_class:
    for food, group in food_instances:
        group_name = str(group).lower()

        # Assign limited foods
        if any(keyword in group_name for keyword in LIMITED_KEYWORDS_DIABETES):
            g.add((diabetes_class, ex.FoodInLimitedAmount, food))

        # Assign high-amount foods
        if any(keyword in group_name for keyword in HIGH_AMOUNT_KEYWORDS_DIABETES):
            g.add((diabetes_class, ex.FoodToBeConsumedInHighAmount, food))

    print(f"✅ Assigned food recommendations to {diabetes_class}")

# Assign food limits to persons with Diabetes
for person, _, health_state in g.triples((None, ex.hasHealthState, None)):
    if str(health_state).endswith("Diabetes"):  # Ensure the health state is Diabetes
        for food, group in food_instances:
            group_name = str(group).lower()

            if any(keyword in group_name for keyword in LIMITED_KEYWORDS_DIABETES):
                g.add((person, ex.FoodInLimitedAmount, food))
            if any(keyword in group_name for keyword in HIGH_AMOUNT_KEYWORDS_DIABETES):
                g.add((person, ex.FoodToBeConsumedInHighAmount, food))

        print(f"✅ Assigned food limits for {person}")

✅ Assigned food recommendations to http://example.com#Diabetes
✅ Assigned food limits for http://example.com#Person_1861
✅ Assigned food limits for http://example.com#Person_4004
✅ Assigned food limits for http://example.com#Person_4304
✅ Assigned food limits for http://example.com#Person_5962


In [32]:
#Rules: FoodInLimitedAmount & FoodToBeConsumedInHighAmount for Hypertension
# Define keyword-based food categories
# Define keyword-based food categories for Hypertension (Matching Food Groups)
LIMITED_KEYWORDS_HYPERTENSION = [
    "salt", "processed", "canned", "snack", "fast food", "fried", 
    "butter", "margarine", "soda", "alcohol", "pickles"
]

HIGH_AMOUNT_KEYWORDS_HYPERTENSION = [
    "low-sodium", "fresh", "no salt", "olive oil", "avocado", "nuts", 
    "whole fruit", "dark chocolate", "water", "herbal tea", 
    "leafy greens", "beets", "bell peppers"
]


# Ensure Hypertension is defined as a subclass of HealthState
RDFS_SUBCLASS = URIRef("http://www.w3.org/2000/01/rdf-schema#subClassOf")
hypertension_class = None

for subclass, _, parent in g.triples((None, RDFS_SUBCLASS, ex.HealthState)):
    if str(subclass).endswith("Hypertension"):
        hypertension_class = subclass
        break

# Retrieve all FoodGroup subclasses
food_groups = set()
for subclass, _, parent in g.triples((None, RDFS_SUBCLASS, ex.FoodGroup)):
    food_groups.add(subclass)

# Find food instances and their FoodGroup assignments
food_instances = set()
for food, _, _ in g.triples((None, RDF.type, None)):
    # Check if this type is a FoodGroup subclass
    for food_group in food_groups:
        if (food, RDF.type, food_group) in g:
            food_instances.add((food, food_group))

# Assign food items based on FoodGroup matching
if hypertension_class:
    for food, group in food_instances:
        group_name = str(group).lower()

        # Assign limited foods
        if any(keyword in group_name for keyword in LIMITED_KEYWORDS_HYPERTENSION):
            g.add((hypertension_class, ex.FoodInLimitedAmount, food))

        # Assign high-amount foods
        if any(keyword in group_name for keyword in HIGH_AMOUNT_KEYWORDS_HYPERTENSION):
            g.add((hypertension_class, ex.FoodToBeConsumedInHighAmount, food))

    print(f"✅ Assigned food recommendations to {hypertension_class}")

# Assign food limits to persons with Hypertension
for person, _, health_state in g.triples((None, ex.hasHealthState, None)):
    if str(health_state).endswith("Hypertension"):  # Ensure the health state is Hypertension
        for food, group in food_instances:
            group_name = str(group).lower()

            if any(keyword in group_name for keyword in LIMITED_KEYWORDS_HYPERTENSION):
                g.add((person, ex.FoodInLimitedAmount, food))
            if any(keyword in group_name for keyword in HIGH_AMOUNT_KEYWORDS_HYPERTENSION):
                g.add((person, ex.FoodToBeConsumedInHighAmount, food))

        print(f"✅ Assigned food limits for {person}")


✅ Assigned food recommendations to http://example.com#Hypertension
✅ Assigned food limits for http://example.com#Person_5950
✅ Assigned food limits for http://example.com#Person_6806


In [33]:
#Rules: FoodInLimitedAmount & FoodToBeConsumedInHighAmount for Pregnancy
# Define keyword-based food categories for Pregnancy (Matching Food Groups)
LIMITED_KEYWORDS_PREGNANCY = [
    "raw meat", "raw fish", "unpasteurized", "deli meat", "sushi", "swordfish", 
    "shark", "king mackerel", "tuna", "coffee", "energy drink", "cola", 
    "soft cheese", "brie", "camembert", "fast food", "processed meat"
]

HIGH_AMOUNT_KEYWORDS_PREGNANCY = [
    "cooked meat", "pasteurized dairy", "well-cooked eggs", "salmon", "sardines", 
    "trout", "decaf coffee", "herbal tea", "hard cheese", "cheddar", "mozzarella",
    "spinach", "lentils", "lean beef"
]

# Ensure Pregnancy is defined as a subclass of HealthState
RDFS_SUBCLASS = URIRef("http://www.w3.org/2000/01/rdf-schema#subClassOf")
pregnancy_class = None

for subclass, _, parent in g.triples((None, RDFS_SUBCLASS, ex.HealthState)):
    if str(subclass).endswith("Pregnancy"):
        pregnancy_class = subclass
        break

# Retrieve all FoodGroup subclasses
food_groups = set()
for subclass, _, parent in g.triples((None, RDFS_SUBCLASS, ex.FoodGroup)):
    food_groups.add(subclass)

# Find food instances and their FoodGroup assignments
food_instances = set()
for food, _, _ in g.triples((None, RDF.type, None)):
    # Check if this type is a FoodGroup subclass
    for food_group in food_groups:
        if (food, RDF.type, food_group) in g:
            food_instances.add((food, food_group))

# Assign food items based on FoodGroup matching
if pregnancy_class:
    for food, group in food_instances:
        group_name = str(group).lower()

        # Assign limited foods
        if any(keyword in group_name for keyword in LIMITED_KEYWORDS_PREGNANCY):
            g.add((pregnancy_class, ex.FoodInLimitedAmount, food))

        # Assign high-amount foods
        if any(keyword in group_name for keyword in HIGH_AMOUNT_KEYWORDS_PREGNANCY):
            g.add((pregnancy_class, ex.FoodToBeConsumedInHighAmount, food))

    print(f"✅ Assigned food recommendations to {pregnancy_class}")

# Assign food limits to persons with Pregnancy
for person, _, health_state in g.triples((None, ex.hasHealthState, None)):
    if str(health_state).endswith("Pregnancy"):  # Ensure the health state is Pregnancy
        for food, group in food_instances:
            group_name = str(group).lower()

            if any(keyword in group_name for keyword in LIMITED_KEYWORDS_PREGNANCY):
                g.add((person, ex.FoodInLimitedAmount, food))
            if any(keyword in group_name for keyword in HIGH_AMOUNT_KEYWORDS_PREGNANCY):
                g.add((person, ex.FoodToBeConsumedInHighAmount, food))

        print(f"✅ Assigned food limits for {person}")

✅ Assigned food recommendations to http://example.com#Pregnancy


In [34]:
#Rule: Foods restricted because of allergy
# Map allergy class names to associated keywords to look for in food labels
allergy_keywords = {
    "Peanut_Allergy": ["peanut"],
    "Milk_Allergy": ["milk"],
    "Eggs_Allergy": ["egg"],
    "TreeNuts_Allergy": ["almond", "cashew", "walnut", "hazelnut", "nuts"],
    "Soy_Allergy": ["soy"],
    "Wheat_Allergy": ["wheat", "barley", "rye"],
    "Fish_Allergy": ["fish", "salmon", "tuna", "cod"],
    "Shellfish_Allergy": ["shrimp", "crab", "lobster"],
}

# Step 3: Define the object property if needed
restricted_food_property = ex.RestrictedFood
g.add((restricted_food_property, RDF.type, OWL.ObjectProperty))
g.add((restricted_food_property, RDFS.label, Literal("RestrictedFood")))

# Step 4: Loop through all food instances and add the restriction
for food_instance, _, _ in g.triples((None, RDF.type, ex.Food)):
    food_uri_str = str(food_instance).lower()

    for allergy_class, keywords in allergy_keywords.items():
        if any(kw in food_uri_str for kw in keywords):
            allergy_uri = ex[allergy_class]
            g.add((food_instance, restricted_food_property, allergy_uri))
            print(f"⚠️ {food_instance} is now a RestrictedFood for {allergy_uri}")


⚠️ http://example.com#Food_almond_imitation_paste_av is now a RestrictedFood for http://example.com#TreeNuts_Allergy
⚠️ http://example.com#Food_almond_paste_filled_tarts_av is now a RestrictedFood for http://example.com#TreeNuts_Allergy
⚠️ http://example.com#Food_almond_paste_filled_tarts_w_butter is now a RestrictedFood for http://example.com#TreeNuts_Allergy
⚠️ http://example.com#Food_almond_paste_filled_tarts_wo_butter is now a RestrictedFood for http://example.com#TreeNuts_Allergy
⚠️ http://example.com#Food_almond_paste_w_egg is now a RestrictedFood for http://example.com#Eggs_Allergy
⚠️ http://example.com#Food_almond_paste_w_egg is now a RestrictedFood for http://example.com#TreeNuts_Allergy
⚠️ http://example.com#Food_almonds_blanched_salted is now a RestrictedFood for http://example.com#TreeNuts_Allergy
⚠️ http://example.com#Food_almonds_blanched_unsalted is now a RestrictedFood for http://example.com#TreeNuts_Allergy
⚠️ http://example.com#Food_almonds_w_skin_salted is now a Rest

In [35]:
#Rule: Foods highly recommended for deficiency
# Step 1: Define the deficiency-to-food-keywords mapping
deficiency_keywords = {
    "Iron_Deficiency": ["spinach", "lentil", "beef", "liver", "tofu", "pumpkin_seed", "chickpea"],
    "VitaminC_Deficiency": ["orange", "citrus", "kiwi", "broccoli", "strawberry", "pepper"],
    "Magnesium_Deficiency": ["almond", "cashew", "spinach", "avocado", "banana", "pumpkin_seed"],
    "Calcium_Deficiency": ["yogurt", "cheese", "milk", "broccoli", "kale", "sardine", "tofu"],
    "VitaminB12_Deficiency": ["egg", "salmon", "tuna", "beef", "milk", "cheese"],
    "VitaminD_Deficiency": ["salmon", "sardine", "egg", "mushroom", "fortified_milk"]
}

# Step 2: Define and declare the HighlyRecommendedFor object property
highly_recommended_for = ex.highlyRecommendedFor
g.add((highly_recommended_for, RDF.type, OWL.ObjectProperty))
g.add((highly_recommended_for, RDFS.label, Literal("HighlyRecommendedFor")))

# Step 3: Loop through food instances and link based on URI keyword match
for food_instance, _, _ in g.triples((None, RDF.type, ex.Food)):
    food_uri_str = str(food_instance).lower()

    for deficiency_class, keywords in deficiency_keywords.items():
        if any(kw in food_uri_str for kw in keywords):
            deficiency_uri = ex[deficiency_class]
            g.add((food_instance, highly_recommended_for, deficiency_uri))
            print(f"✅ {food_instance} is now HighlyRecommendedFor {deficiency_uri}")


✅ http://example.com#Food_almond_imitation_paste_av is now HighlyRecommendedFor http://example.com#Magnesium_Deficiency
✅ http://example.com#Food_almond_paste_filled_tarts_av is now HighlyRecommendedFor http://example.com#Magnesium_Deficiency
✅ http://example.com#Food_almond_paste_filled_tarts_w_butter is now HighlyRecommendedFor http://example.com#Magnesium_Deficiency
✅ http://example.com#Food_almond_paste_filled_tarts_wo_butter is now HighlyRecommendedFor http://example.com#Magnesium_Deficiency
✅ http://example.com#Food_almond_paste_w_egg is now HighlyRecommendedFor http://example.com#Magnesium_Deficiency
✅ http://example.com#Food_almond_paste_w_egg is now HighlyRecommendedFor http://example.com#VitaminB12_Deficiency
✅ http://example.com#Food_almond_paste_w_egg is now HighlyRecommendedFor http://example.com#VitaminD_Deficiency
✅ http://example.com#Food_almonds_blanched_salted is now HighlyRecommendedFor http://example.com#Magnesium_Deficiency
✅ http://example.com#Food_almonds_blanche

In [36]:
# URI of the Pregnancy health state
pregnancy = ex.Pregnancy

# Go through all persons with hasHealthState Pregnancy
for person, _, state in g.triples((None, ex.hasHealthState, pregnancy)):
    sex = g.value(person, ex.hasSex, None)

    # Check if the person has sex explicitly set and is not Female
    if sex and str(sex) != "Female":
        print(f"⚠️ Person {person} is {sex} and cannot be assigned Pregnancy.")
        # Optionally: remove the invalid triple
        g.remove((person, ex.hasHealthState, pregnancy))

In [37]:
# Save the updated ontology
updated_ttl_file_path = "Updated_Food_Health_sustainability_with_Rules_V2.ttl"
g.serialize(destination=updated_ttl_file_path, format="turtle")

print(f"Updated ontology saved to {updated_ttl_file_path}")

Updated ontology saved to Updated_Food_Health_sustainability_with_Rules_V2.ttl
